In [12]:
# Anil Kumar Ravuru

import pandas as pd

df = pd.read_csv('english_posts.csv')
df.head()

,name,description,message,fb_like,fb_angry,fb_thankful,fb_haha,fb_sad,fb_wow,fb_love,top_reaction_count,top_reaction
0,This Korean Ad Hilariously Addresses The Power...,But how is this about the burger?,A proof that makeup and sorcery go hand in han...,1878,1,0,391,0,3,22,391,fb_haha
1,People Are Losing It For The Hiddle-Swift Split,Click to see the pic and write a comment...,My imaginary relationship with Tom Hiddleston ...,7606,5,0,1629,8,13,269,1629,fb_haha
2,"Sia Drops ""The Greatest"" Video Featuring 49 Da...",49 young dancers representing the 49 people lo...,Another amazing and breathtaking video from Si...,25304,6,0,95,44,367,2065,2065,fb_love
3,15 Tweets That Remind You Why You Need A Condo...,Click to see the pic and write a comment...,They also remind you how great your parents ar...,3452,2,0,607,0,1,62,607,fb_haha
4,Russian Topless Women Help Stop Speeding Drive...,Is it the breast idea ever?,"Not sure of if it will help, or cause more acc...",6454,12,0,1245,8,93,162,1245,fb_haha


In [13]:
import re
import nltk
    
# Need to improve this grammar to get accurate tokens
# Try to catch smilies
grammar = r"""[a-zA-Z]+(?:[.@$-'*][a-zA-Z]+)*"""

tokenizer = nltk.tokenize.RegexpTokenizer(grammar)
tokenizer.tokenize("Hi, How are you? U.S.A is doing 'great'. Isn't it. 2020 is f@cking with my self-respect :(")    

['Hi',
 'How',
 'are',
 'you',
 'U.S.A',
 'is',
 'doing',
 'great',
 "Isn't",
 'it',
 'is',
 'f@cking',
 'with',
 'my',
 'self',
 'respect']

In [14]:
bad_words = set()
bad_df = pd.read_csv('bad_words.csv',sep='\n', header=None, names=['word'])
for word in bad_df['word']:
    if ' ' not in word:
        bad_words.add(word)
bad_words

def tokenize_words(text):
    global bad_words
    global tokenizer
    tokens = tokenizer.tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] in bad_words:
            tokens[i] = 'badword'
    return ' '.join(tokens).lower()

df['token_words'] = df['name'].apply(lambda text: tokenize_words(text))

In [15]:
df.head()

,name,description,message,fb_like,fb_angry,fb_thankful,fb_haha,fb_sad,fb_wow,fb_love,top_reaction_count,top_reaction,token_words
0,This Korean Ad Hilariously Addresses The Power...,But how is this about the burger?,A proof that makeup and sorcery go hand in han...,1878,1,0,391,0,3,22,391,fb_haha,this korean ad hilariously addresses the power...
1,People Are Losing It For The Hiddle-Swift Split,Click to see the pic and write a comment...,My imaginary relationship with Tom Hiddleston ...,7606,5,0,1629,8,13,269,1629,fb_haha,people are losing it for the hiddle swift split
2,"Sia Drops ""The Greatest"" Video Featuring 49 Da...",49 young dancers representing the 49 people lo...,Another amazing and breathtaking video from Si...,25304,6,0,95,44,367,2065,2065,fb_love,sia drops the greatest video featuring dancers...
3,15 Tweets That Remind You Why You Need A Condo...,Click to see the pic and write a comment...,They also remind you how great your parents ar...,3452,2,0,607,0,1,62,607,fb_haha,tweets that remind you why you need a condom b...
4,Russian Topless Women Help Stop Speeding Drive...,Is it the breast idea ever?,"Not sure of if it will help, or cause more acc...",6454,12,0,1245,8,93,162,1245,fb_haha,russian topless women help stop speeding drive...


In [17]:
# To expand short forms like I'll to I will
# TODO: pycontractions 1.0.1 check this

con_df = pd.read_csv('english_short_forms.csv')[['0','1']]
con_dict = {}
for index, row in con_df.iterrows():
    con_dict[row['0']] = row['1']
def expand_forms(text):
    global con_dict
    for x in con_dict:
        if x in text:
            text = text.replace(x, con_dict[x])
    return text

df['token_words'] = df['token_words'].apply(lambda text: expand_forms(text))

In [18]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
stop_words = nltk.corpus.stopwords.words('english')
def lemmatize_text(text):
    global lemmatizer
    global stemmer
    final_words = []
    for word in text.split(' '):
        if word not in stop_words:
            final_words.append(stemmer.stem((lemmatizer.lemmatize(word))))
    return ' '.join(final_words)

df['token_words'] = df['token_words'].apply(lambda row: lemmatize_text(row))


In [19]:
df.to_csv('Final.csv', index=False)

In [20]:
df.columns

Index(['name', 'description', 'message', 'fb_like', 'fb_angry', 'fb_thankful',
       'fb_haha', 'fb_sad', 'fb_wow', 'fb_love', 'top_reaction_count',
       'top_reaction', 'token_words'],
      dtype='object')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each